In [2]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import sys
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    classification_report, 
    confusion_matrix
)
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
import json
from data_loader import (
    data_loader,
    binned_distr,
    dev_mad_var,
    features_accelerometer,
    features_cosine,
    features_freq,
    features_temporal,
    vector_magnitude,
    peak_features,
)


In [3]:
data_processor_old = data_loader.TimeWindowSegmenter(
    df_path="data_loader/real_world_2016.parquet",
    window_size=10,
    step_size=4,
    source_sampling_rate=50,
    time_column="timestamp",
    id_column='person_id',
    activity_column='activity_label',
    clean_columns=False,
    fix_timestamps=False,
    acc_columns=('acc_x', 'acc_y', 'acc_z'),
    gyr_columns=('gyr_x', 'gyr_y', 'gyr_z')
)
data_processor_old.resample_to(25)

Resampling: 100%|██████████| 71/71 [00:03<00:00, 21.65it/s]


In [4]:
data_processor = data_loader.TimeWindowSegmenter(
    window_size=10,
    step_size=2,
    time_column="timestamp",
    id_column='person_id',
    activity_column='activity_label',
    clean_columns=False,
    fix_timestamps=False,
    acc_columns=('acc_x', 'acc_y', 'acc_z'),
    gyr_columns=('gyr_x', 'gyr_y', 'gyr_z')
)

In [5]:
with open("data_dump/wynik.json", "r") as f:
    raw_data = json.load(f)

In [6]:
fs = 25
rows = []
for session in raw_data["sensors"]:
    session_id = f"person_{session['id']}"
    date = 0
    for sample in session["samples"]:
        timestamp = pd.Timestamp('1970-01-01') + pd.Timedelta(milliseconds=date)
        date+=1000/fs
        label = sample.get("label")
        acc = sample.get("acceleration", [None, None, None])
        gyr = sample.get("gyroscope", [None, None, None])
        wpis = {
            "person_id": session_id,
            "timestamp": timestamp,
            "activity_label": label,
            "acc_x": acc[0] if len(acc) > 0 else None,
            "acc_y": acc[1] if len(acc) > 1 else None,
            "acc_z": acc[2] if len(acc) > 2 else None,
            "gyr_x": gyr[0] if len(gyr) > 0 else None,
            "gyr_y": gyr[1] if len(gyr) > 1 else None,
            "gyr_z": gyr[2] if len(gyr) > 2 else None,
            }
        rows.append(wpis)
data_processor.df = pd.DataFrame(rows)

In [7]:
data_processor_old.df = data_processor_old.df[~data_processor_old.df["activity_label"].isin(["standing", "climbingup"])]

In [8]:
data_processor_old.df["activity_label"].value_counts()

activity_label
running    258467
walking    243150
sitting    240887
Name: count, dtype: int64

In [9]:
data_processor.df["activity_label"].value_counts()

activity_label
sitting    117715
walking     37467
running      7252
Name: count, dtype: int64

In [10]:
def extract_features_from_window(window, fs=20, axes = ['ac_x', 'ac_y', 'ac_z', 'g_x', 'g_y', 'g_z']):
    features = []
    feature_names = []
    data_freq_dict = {}
    
    freq_funcs = [
        ('dom_freq', features_freq.dominant_frequency, [fs]),    
        ('entropy', features_freq.spectral_entropy, [fs]),       
        ('energy', features_freq.spectral_energy, []),           
        ('centroid', features_freq.spectral_centroid, [fs]),     
        ('bandwidth', features_freq.spectral_bandwidth, [fs]),   
        ('flatness', features_freq.spectral_flatness, [fs]),     
        ('slope', features_freq.spectral_slope, [fs]),           
        ('rolloff', features_freq.spectral_rolloff, [fs]),       
        ('band_ratio', features_freq.band_energy_ratio, [fs])    
    ]
    
    for axis in axes:
        signal = window[axis].astype(float).values
        for fname, func, extra_args in freq_funcs:
            data_freq_dict[f"{axis}_{fname}"] = func(signal, *extra_args)
            # features.append(func(signal, *extra_args))
            # feature_names.append(f"{axis}_{fname}")

    # rozkład wartości względnie dla okna
    data_binned_all_dict = binned_distr.calculate_binned_distribution_multi_axis(
        window=window, 
        bins=10,
        axes=axes
    )

    data_binned_sep_dict = {f"{key}_bin{bin_id}":data for key, items in data_binned_all_dict.items() for bin_id, data in enumerate(items)}

    dev_mad_var_dict = dev_mad_var.calculate_statistics_multi_axis(
        window=window,
        axes=axes
    )
    
    acc_features_dict = features_accelerometer.extract_acc_features(
        window=window,
        axes=axes
    )

    cosine_features_dict = features_cosine.extract_cosine_distances(
        window=window,
        axes=axes
    )

    temporal_features_dict = features_temporal.extract_temporal_features(
        window=window,
        axes=axes[3:]
    )

    vector_magnitude_dict = {}
    vector_magnitude_dict["vector_acc_mag"] = vector_magnitude.calculate_accelerometer_magnitude(
        window=window,
        axes=axes[:3]
    )
    vector_magnitude_dict["vector_gyr_mag"] = vector_magnitude.calculate_gyroscope_magnitude(
        window=window,
        axes=axes[3:]
    )

    peak_features_dict = peak_features.extract_peak_features(
        window_df=window,
        sampling_rate=fs,
        axes=axes
    )

    
    return {**data_freq_dict, **data_binned_sep_dict, **dev_mad_var_dict, **acc_features_dict, **cosine_features_dict, **temporal_features_dict, **vector_magnitude_dict, **peak_features_dict}

In [11]:
X_OLD = []
Y_OLD =[]
feature_names = None
for window in data_processor_old.segment():
        label = window['activity_label'].mode()[0]
        if label == "climbingup":
                continue
        results = extract_features_from_window(
                window,
                fs=50,
                axes=['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z']
        )
        
        X_OLD.append(results)
        Y_OLD.append(label)

Segmenting:  98%|█████████▊| 45/46 [00:25<00:00,  1.74it/s]


In [12]:
X_OLD_df = pd.DataFrame(X_OLD)
X_OLD_df

,acc_x_dom_freq,acc_x_entropy,acc_x_energy,acc_x_centroid,acc_x_bandwidth,acc_x_flatness,acc_x_slope,acc_x_rolloff,acc_x_band_ratio,acc_y_dom_freq,...,peak_count_acc_z,peak_avg_time_diff_gyr_x,peak_std_time_diff_gyr_x,peak_count_gyr_x,peak_avg_time_diff_gyr_y,peak_std_time_diff_gyr_y,peak_count_gyr_y,peak_avg_time_diff_gyr_z,peak_std_time_diff_gyr_z,peak_count_gyr_z
0,2.5,0.301576,14221.114947,2.996298,2.263453,0.037393,-0.593347,2.6,33.136844,5.0,...,85,0.085345,0.027210,117,0.154687,0.060853,65,0.161695,0.089313,60
1,2.5,0.298869,10621.561723,3.041746,2.468296,0.046304,-0.499171,2.5,45.799751,4.9,...,73,0.088036,0.028089,113,0.145970,0.055181,68,0.181111,0.108007,55
2,2.4,0.285303,10504.366442,3.064647,2.450723,0.030720,-0.553173,2.5,35.521586,4.8,...,67,0.090741,0.030963,109,0.141714,0.055317,71,0.189231,0.111938,53
3,2.4,0.307100,10653.840249,3.113361,2.593034,0.031651,-0.507251,2.5,33.738230,4.8,...,57,0.084444,0.030282,118,0.143768,0.055979,70,0.189804,0.110214,52
4,2.4,0.335691,12467.593422,3.054048,2.496021,0.042499,-0.750923,2.5,40.742747,4.7,...,62,0.081833,0.029552,121,0.145970,0.055397,68,0.205957,0.117919,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,3.3,0.562343,47774.734831,3.520897,2.827273,0.076215,-0.922476,3.9,19.308484,1.6,...,60,0.172364,0.237670,56,0.186275,0.226360,52,0.217209,0.253004,44
3596,3.3,0.580853,47585.537627,2.808030,2.950003,0.063462,-0.951745,3.5,23.797326,1.7,...,62,0.170690,0.233540,59,0.179630,0.221393,55,0.235610,0.255229,42
3597,3.4,0.393745,50074.012927,4.273206,2.927802,0.050638,-0.613791,5.1,11.163659,0.1,...,85,0.125333,0.076548,76,0.145000,0.061135,65,0.205000,0.080128,45
3598,3.4,0.567407,49134.988011,4.759545,3.359068,0.115590,-0.755875,8.2,7.585431,0.2,...,101,0.100000,0.103055,99,0.127273,0.114774,78,0.175636,0.139906,56


In [13]:
X = []
Y =[]
feature_names = None
for window in data_processor.segment():
        label = window['activity_label'].mode()[0]
        if label == "climbingup":
                continue
        results = extract_features_from_window(
                window,
                fs=50,
                axes=['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z']
        )
        
        X.append(results)
        Y.append(label)

Segmenting: 100%|██████████| 223/223 [00:20<00:00, 10.89it/s]


In [14]:
X_df = pd.DataFrame(X)
X_df

,acc_x_dom_freq,acc_x_entropy,acc_x_energy,acc_x_centroid,acc_x_bandwidth,acc_x_flatness,acc_x_slope,acc_x_rolloff,acc_x_band_ratio,acc_y_dom_freq,...,peak_count_acc_z,peak_avg_time_diff_gyr_x,peak_std_time_diff_gyr_x,peak_count_gyr_x,peak_avg_time_diff_gyr_y,peak_std_time_diff_gyr_y,peak_count_gyr_y,peak_avg_time_diff_gyr_z,peak_std_time_diff_gyr_z,peak_count_gyr_z
0,1.00,0.883737,0.002737,10.060141,6.853524,0.496969,-0.116080,18.50,1.412742,0.50,...,57,0.067719,0.025341,58,0.070741,0.026307,55,0.074510,0.023789,52
1,7.00,0.902650,0.002757,9.987085,6.583016,0.557020,-0.216642,18.25,1.683527,0.50,...,58,0.071636,0.027684,56,0.075294,0.028721,52,0.070000,0.021381,57
2,6.75,0.897319,0.002768,10.078286,6.478964,0.502360,-0.311404,18.25,1.731866,0.50,...,59,0.076863,0.026383,52,0.076400,0.030382,51,0.073208,0.024860,54
3,3.75,0.908680,0.002733,9.110566,6.571613,0.528826,-0.368858,18.00,2.036705,2.25,...,58,0.075294,0.026148,52,0.075686,0.030503,52,0.070182,0.025262,56
4,3.75,0.898120,0.002722,9.601411,6.874643,0.531381,-0.277630,18.50,1.620274,9.00,...,58,0.078776,0.026313,50,0.071852,0.028420,55,0.070000,0.030732,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3009,3.50,0.801546,2.578696,5.448570,3.749124,0.202127,-1.070797,10.00,5.718880,0.25,...,65,0.073846,0.026758,53,0.086818,0.038834,45,0.090698,0.036941,44
3010,3.50,0.807620,2.462039,5.108070,3.776047,0.198910,-1.103520,9.25,6.500000,0.25,...,61,0.083404,0.032042,48,0.089767,0.035666,44,0.083913,0.030821,47
3011,8.75,0.854977,2.417639,7.940028,5.834716,0.409984,-0.537718,13.00,2.449982,0.25,...,61,0.084000,0.032551,46,0.087727,0.031685,45,0.081667,0.032102,49
3012,9.50,0.871887,2.189855,11.120433,5.562183,0.503664,-0.129245,18.00,1.605844,0.25,...,61,0.089767,0.033791,44,0.083478,0.030160,47,0.079592,0.028994,50


## trenowanie modelu z danych zebranych ręcznie

In [73]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_df, Y,             # data frame razem z cechami i etykietami
    test_size=0.2,       # 20% walidacja
    random_state=42,     # dla powtarzalności
    stratify=Y           # zachowanie proporcji klas
)

# utworzenie i trenowanie modelu
model = SVC(kernel='rbf', C=1.0, gamma='scale')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
# Obliczenie dokładności
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_val, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_val, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.99
Confusion Matrix:
[[ 26   0   1]
 [  0 433   1]
 [  1   4 137]]
Classification Report:
              precision    recall  f1-score   support

     running       0.96      0.96      0.96        27
     sitting       0.99      1.00      0.99       434
     walking       0.99      0.96      0.98       142

    accuracy                           0.99       603
   macro avg       0.98      0.98      0.98       603
weighted avg       0.99      0.99      0.99       603



In [50]:
len(Y_OLD)

3600

## Trenowanie modelu z danych benchmarkowych


In [68]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_OLD_df, Y_OLD,             # data frame razem z cechami i etykietami
    test_size=0.2,       # 20% walidacja
    random_state=42,     # dla powtarzalności
    stratify=Y_OLD           # zachowanie proporcji klas
)

# utworzenie i trenowanie modelu
model_old = SVC(kernel='rbf', C=1.0, gamma='scale')
model_old.fit(X_train, Y_train)
Y_pred = model_old.predict(X_val)
# Obliczenie dokładności
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_val, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_val, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.93
Confusion Matrix:
[[214  15  22]
 [  0 232   1]
 [  0  10 226]]
Classification Report:
              precision    recall  f1-score   support

     running       1.00      0.85      0.92       251
     sitting       0.90      1.00      0.95       233
     walking       0.91      0.96      0.93       236

    accuracy                           0.93       720
   macro avg       0.94      0.94      0.93       720
weighted avg       0.94      0.93      0.93       720



## Model trenowany na danych benchmarkowych ale sprawdzony na danych zebranych ręcznie.

In [75]:
X_manual, _, Y_manual, _ = train_test_split(
    X_df, Y,             # data frame razem z cechami i etykietami
    test_size=1,       # 20% walidacja
)
Y_pred = model_old.predict(X_manual)
accuracy = accuracy_score(Y_manual, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_manual, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_manual, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.72
Confusion Matrix:
[[   0  133    0]
 [   0 2168    0]
 [  10  702    0]]
Classification Report:
              precision    recall  f1-score   support

     running       0.00      0.00      0.00       133
     sitting       0.72      1.00      0.84      2168
     walking       0.00      0.00      0.00       712

    accuracy                           0.72      3013
   macro avg       0.24      0.33      0.28      3013
weighted avg       0.52      0.72      0.60      3013



d:\metamotion-ml\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\metamotion-ml\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\metamotion-ml\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Model z danych zebranych ale sprawdzony na danych benchmarkowych

```python

In [74]:
X_old, _, Y_old, _ = train_test_split(
    X_OLD_df, Y_OLD,             # data frame razem z cechami i etykietami
    test_size=1,       # 20% walidacja
)
Y_pred = model.predict(X_old)
accuracy = accuracy_score(Y_old, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_old, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_old, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.33
Confusion Matrix:
[[   0    0 1255]
 [   0    0 1166]
 [   0    0 1178]]
Classification Report:
              precision    recall  f1-score   support

     running       0.00      0.00      0.00      1255
     sitting       0.00      0.00      0.00      1166
     walking       0.33      1.00      0.49      1178

    accuracy                           0.33      3599
   macro avg       0.11      0.33      0.16      3599
weighted avg       0.11      0.33      0.16      3599



d:\metamotion-ml\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\metamotion-ml\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\metamotion-ml\.venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Model z danych hybrydowych

In [17]:
X_merged = pd.concat([X_OLD_df, X_df], ignore_index=True)
Y_merged = [*Y_OLD, *Y]


In [100]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_merged, Y_merged,             # data frame razem z cechami i etykietami
    test_size=0.2,       # 20% walidacja
    random_state=42,     # dla powtarzalności
    stratify=Y_merged           # zachowanie proporcji klas
)

# utworzenie i trenowanie modelu
model_hybrid = SVC(kernel='rbf', C=1.0, gamma='scale')
model_hybrid.fit(X_train, Y_train)
Y_pred = model_hybrid.predict(X_val)
# Obliczenie dokładności
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_val, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_val, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.95
Confusion Matrix:
[[234  13  31]
 [  0 660   7]
 [  2  13 363]]
Classification Report:
              precision    recall  f1-score   support

     running       0.99      0.84      0.91       278
     sitting       0.96      0.99      0.98       667
     walking       0.91      0.96      0.93       378

    accuracy                           0.95      1323
   macro avg       0.95      0.93      0.94      1323
weighted avg       0.95      0.95      0.95      1323



## test modelu hybrydowego na tylko danych zebranych ręcznie

In [101]:
X_manual, _, Y_manual, _ = train_test_split(
    X_df, Y,             # data frame razem z cechami i etykietami
    test_size=1,       # 20% walidacja
)
Y_pred = model_hybrid.predict(X_manual)
accuracy = accuracy_score(Y_manual, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_manual, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_manual, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.97
Confusion Matrix:
[[  86    0   47]
 [   0 2145   24]
 [   9   10  692]]
Classification Report:
              precision    recall  f1-score   support

     running       0.91      0.65      0.75       133
     sitting       1.00      0.99      0.99      2169
     walking       0.91      0.97      0.94       711

    accuracy                           0.97      3013
   macro avg       0.94      0.87      0.90      3013
weighted avg       0.97      0.97      0.97      3013



In [102]:
import joblib

# Zapis modelu do pliku
joblib.dump(model, 'svc_model_manual.pkl')
joblib.dump(model_hybrid, 'svc_model_hybrid.pkl')


['svc_model_hybrid.pkl']

# Podejście z Normalizacją danych

## trenowanie modelu z danych benchmarkowych ale z normalizowanych. 

In [15]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_OLD_df, Y_OLD,
    test_size=0.2,
    random_state=42,
    stratify=Y_OLD
)


# Pipeline: normalizacja + model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', C=1.0, gamma='scale'))
])

pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_val)

accuracy = accuracy_score(Y_val, Y_pred)
print(f'Accuracy (with normalization): {accuracy:.2f}')
conf_matrix = confusion_matrix(Y_val, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)
class_report = classification_report(Y_val, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy (with normalization): 0.97
Confusion Matrix:
[[237  11   3]
 [  1 232   0]
 [  2   1 233]]
Classification Report:
              precision    recall  f1-score   support

     running       0.99      0.94      0.97       251
     sitting       0.95      1.00      0.97       233
     walking       0.99      0.99      0.99       236

    accuracy                           0.97       720
   macro avg       0.98      0.98      0.98       720
weighted avg       0.98      0.97      0.97       720



## sprawdzenie modelu na danych benchmarkowych korzystajac z normalizacji

In [16]:
X_manual, _, Y_manual, _ = train_test_split(
    X_df, Y,             # data frame razem z cechami i etykietami
    test_size=1,       # 20% walidacja
)
Y_pred = pipeline.predict(X_manual)

accuracy = accuracy_score(Y_manual, Y_pred)
print(f'Accuracy (with normalization): {accuracy:.2f}')
conf_matrix = confusion_matrix(Y_manual, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)
class_report = classification_report(Y_manual, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy (with normalization): 0.69
Confusion Matrix:
[[   0  133    0]
 [  88 2078    2]
 [   0  712    0]]
Classification Report:
              precision    recall  f1-score   support

     running       0.00      0.00      0.00       133
     sitting       0.71      0.96      0.82      2168
     walking       0.00      0.00      0.00       712

    accuracy                           0.69      3013
   macro avg       0.24      0.32      0.27      3013
weighted avg       0.51      0.69      0.59      3013



## Model Hybrydowy z normalizacją

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(
    X_merged, Y_merged,             # data frame razem z cechami i etykietami
    test_size=0.2,       # 20% walidacja
    random_state=42,     # dla powtarzalności
    stratify=Y_merged           # zachowanie proporcji klas
)

# utworzenie i trenowanie modelu
pipeline_hybrid = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', C=1.0, gamma='scale'))
])


pipeline_hybrid.fit(X_train, Y_train)
Y_pred = pipeline_hybrid.predict(X_val)
# Obliczenie dokładności
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_val, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_val, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 0.99
Confusion Matrix:
[[265  11   2]
 [  2 665   0]
 [  0   1 377]]
Classification Report:
              precision    recall  f1-score   support

     running       0.99      0.95      0.97       278
     sitting       0.98      1.00      0.99       667
     walking       0.99      1.00      1.00       378

    accuracy                           0.99      1323
   macro avg       0.99      0.98      0.99      1323
weighted avg       0.99      0.99      0.99      1323



### test hybrydowego modelu na danych ręcznie zebranych

In [19]:
X_manual, _, Y_manual, _ = train_test_split(
    X_df, Y,             # data frame razem z cechami i etykietami
    test_size=1,       # 20% walidacja
)
Y_pred = pipeline_hybrid.predict(X_manual)
accuracy = accuracy_score(Y_manual, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Wyświetlenie macierzy konfuzji
conf_matrix = confusion_matrix(Y_manual, Y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Wyświetlenie szczegółowego raportu klasyfikacji
class_report = classification_report(Y_manual, Y_pred)
print("Classification Report:")
print(class_report)

Accuracy: 1.00
Confusion Matrix:
[[ 133    0    0]
 [   0 2167    1]
 [   0    1  711]]
Classification Report:
              precision    recall  f1-score   support

     running       1.00      1.00      1.00       133
     sitting       1.00      1.00      1.00      2168
     walking       1.00      1.00      1.00       712

    accuracy                           1.00      3013
   macro avg       1.00      1.00      1.00      3013
weighted avg       1.00      1.00      1.00      3013



In [20]:
import joblib

# Zapis modelu do pliku
joblib.dump(pipeline_hybrid, 'svc_model_norm_hybrid.joblib')

['svc_model_norm_hybrid.joblib']